In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title IMPORTS
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sbn

import zipfile
import random as rd
import numpy as np
import pandas as pd
import os, signal
from IPython.display import clear_output
def clear(): clear_output()

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
#from tensorflow.keras.preprocessing.image import img_to_array, load_img
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, InputLayer, Input
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam

In [ ]:
#Load
mnist = tf.keras.datasets.mnist
(imgs_train, ans_train), (imgs_test, ans_test) = mnist.load_data()

# Knobs
nrows = 2
ncols = 5
N = 10

# Data
imgs_train.shape, imgs_test.shape

In [ ]:
    im = np.zeros((28,56))
    tens_index = rd.randint(0,len(ans_train)-1)   
    ones_index = rd.randint(0,len(ans_train)-1) 
    for i in range(28):
        im[:,i] = imgs_train[tens_index,:,i]
        im[:,i+28] = imgs_train[ones_index,:,i]
    answer = ans_train[tens_index]*10 + ans_train[ones_index]
    print ("Answer =", ans_train[tens_index],"* 10 +",ans_train[ones_index],"=",answer)
    plt.imshow(im)

In [ ]:
# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [ ]:
###################### Stack Model Layers
##########################################
def stack_new_model():
    input_layer = layers.Input(shape=(28, 56, 1))
    x = layers.Conv2D(4, 6, activation='relu')(input_layer)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(3, 3, activation='relu', strides=(2))(x) 
    x = layers.Conv2D(1, 3, activation='relu', strides=(2))(x) 
    #x = layers.Conv2D(1, 2, activation='relu', strides=(1))(x) 
    #x = layers.MaxPooling2D(2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(12, activation='relu')(x)  ### how few ?! ####
    x = layers.Dropout(0.1)(x)
    output_layer = layers.Dense(100)(x)#, activation='softmax')(x)
    model = Model(input_layer, output_layer)
    ############################## Compile
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                                                            optimizer=Adam(lr=0.0075),
                                                            metrics=['acc'])
    return model, ([], [], [], []) # new set each new _compile_

In [ ]:
####################### TF Datasets for input

train_ds = tf.data.Dataset.from_tensor_slices( (x_train, y_train) )
test_ds = tf.data.Dataset.from_tensor_slices( (x_test, y_test) )

###################### Stack Layers
input_layer = layers.Input(shape=(28, 56, 1))
x = layers.Conv2D(2, (3,4), activation='relu')(input_layer)
x = layers.MaxPooling2D(2)(x)
#x = layers.Conv2D(3, (2,3), activation='relu')(x) 
#x = layers.MaxPooling2D(2)(x)
#x = layers.Conv2D(5, (2,3), activation='relu')(x) # ,strides=(2))(x) 
x = layers.Flatten()(x)
#x = layers.Dense(10, activation='relu')(x) 
x = layers.Dropout(0.25)(x)
output_layer = layers.Dense(100)(x)#, activation='softmax')(x)

####################### Build
model = Model(input_layer, output_layer)

####################### Compile
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=Adam(lr=0.0075),
                metrics=['acc'])
####################### Layer-outputs model
layer_outputs = [layer.output for layer in model.layers[1:]]
output_model = Model(input_layer, layer_outputs)

####################### History containers
answers, guesses = [],[]#, max_guesses, flatout = [],[],[],np.zeros((N,4))

In [ ]:
############################# Train
mhistory = model.fit(train_ds.shuffle(40001).batch(40), 
                        epochs=15,  
                        validation_data=test_ds.batch(100), 
                        verbose=1)

In [ ]:
################ Save / Load weights and ckpts
###model.load_weights('VCD_weights/VCD_weights')
###model.save_weights('VCD_weights/VCD_weights')

In [ ]:
#@title Function GUESSING()
def guessing(N=1):
    answers, guesses = [],[]
    print ('Processing...\n')
    for count in range(N):
        take1_ds = test_ds.shuffle(10001).take(1)
        for img, ans in take1_ds:
            answer = ans.numpy()
            img = img.numpy()                  # 28x56x1
        img = img.reshape((1,) + img.shape)    # 1x28x56x1
        guess_set = model.predict(img)
        guess = tf.random.categorical( guess_set, num_samples=1 ).numpy().squeeze()

        answers += [answer]
        guesses += [guess]
        #pG += [guess_set[guess]]
        #pA += [guess_set[answer]]
        #print("Answer",answer,"\tGuess",guess)#, "\tp(A)",pA,"\tp(G)",pG)
       
        if count%20 == 0:
            print ('Processing...',count,"...")
            clear_output(wait=True)

    return answers, guesses